In [1]:
import os, sys

# Change working directory to project root (two levels up from /src/training/)
os.chdir(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))
sys.path.append(os.getcwd())

In [2]:
import numpy as np
from src.core.config import MAXIMUM_MOVES, DETECTIVE_INITIAL_TICKETS, MRX_INITIAL_TICKETS, INITIAL_NUMBER_OF_2X_TICKETS
from src.core.transport import TransportType, Ticket
from src.core.board import Board

## Random games generation

In [3]:
board = Board(board_data_filePath="src/core/data/board.txt")

possible_initial_positions = []
with open("src/core/data/initial_positions.txt", 'r') as file:
    possible_initial_positions = [int(line.strip()) for line in file]

In [12]:
def get_detective_available_transport_types(detective_number: int, current_position: int, tickets):
    result = []
    available_transport_types = board.get_possible_transport_types(current_position)

    if (tickets[3*detective_number] > 0) and TransportType.TAXI in available_transport_types:
        result.append([TransportType.TAXI, tickets[3*detective_number]])

    if (tickets[3*detective_number+1] > 0) and TransportType.BUS in available_transport_types:
        result.append([TransportType.BUS, tickets[tickets[3*detective_number+1]]])

    if (tickets[3*detective_number+2] > 0) and TransportType.UNDERGROUND in available_transport_types:
        result.append([TransportType.UNDERGROUND, tickets[3*detective_number+2]])

    return np.array(result)

def generate_single_game(num_detectives: int, num_moves: int):
    winner = -1 # 1 = detective won; -1 = MrX won.
    positions = np.zeros((num_moves, num_detectives+1), dtype=int) # Columns are n detectives + Mrx
    # Columns are D1 Taxi, D1 Bus, D1 Underground, D2 Taxi, ..., Dn Underground, Mrx Black, Mrx 2x
    tickets = np.zeros((num_moves, 3*num_detectives+2), dtype=int) 

    # 1. Start game with random positions.
    random_positions = np.random.choice(possible_initial_positions, size=n + 1, replace=False)
    positions[0, :] = random_positions
    initial_tickets = [DETECTIVE_INITIAL_TICKETS[TransportType.TAXI], DETECTIVE_INITIAL_TICKETS[TransportType.BUS], DETECTIVE_INITIAL_TICKETS[TransportType.UNDERGROUND]] * num_detectives + [MRX_INITIAL_TICKETS[TransportType.BLACK], INITIAL_NUMBER_OF_2X_TICKETS]
    tickets[0, :] = np.array(initial_tickets)

    for m in range(num_moves - 1):
        tickets[m+1, :] = tickets[m, :]

        # 2. For each detective randomly pick any of possible ticket type and then get random possible next possition.
        for i in range(num_detectives):
            available_transport_types = get_detective_available_transport_types(i, positions[m, i], tickets[m, :])

            if len(available_transport_types) == 0 or np.sum(available_transport_types[:, 1]) == 0:
                break
            
            # Selecting tickets with probabilities higher for tickets that are more. This can be revisited to update
            # based on the results, if required.
            probabilities = 100  * available_transport_types[:, 1] // np.sum(available_transport_types[:, 1])

            # Making sum of probabilities to 100 by adding remaining to the smallest. The sum will never be grater than 100 as we
            # are taking floor while dividing
            if np.sum(probabilities) < 100:
                probabilities[np.argmin(probabilities)] += (100 - np.sum(probabilities))

            chosen_transport_type = np.random.choice(available_transport_types[:, 0], p=[i/100 for i in probabilities])

            possible_positions = board.get_possible_destinations(positions[m, i], chosen_transport_type, False)
            chosen_position = np.random.choice(possible_positions)

            # Update positions and tickets in the results
            positions[m+1, i] = chosen_position
            if chosen_transport_type == TransportType.TAXI:
                tickets[m+1, 3*i] -= 1
            elif chosen_transport_type == TransportType.BUS:
                tickets[m+1, 3*i + 1] -= 1
            elif chosen_transport_type == TransportType.UNDERGROUND:
                tickets[m+1, 3*i + 2] -= 1

        # 3. For MrX, additionally get probability of using 2X ticket.
        chosen_transport_type: TransportType
        available_transport_types = board.get_possible_transport_types(positions[m, -1])

        is_black_ticket_available: bool = tickets[m, -2] > 0
        should_use_black_ticket_as_mask: bool = False

        if is_black_ticket_available and TransportType.BLACK in available_transport_types:
            if len(available_transport_types) == 1 or np.random.choice([True, False]):
                chosen_transport_type = TransportType.BLACK
            else:
                chosen_transport_type = np.random.choice([t for t in available_transport_types if t != TransportType.BLACK])
        else:
            chosen_transport_type = np.random.choice(available_transport_types)
            if is_black_ticket_available:
                # To represent the case where black ticket is used as mask for actual transport.
                should_use_black_ticket_as_mask = np.random.choice([True, False], p=[0.4, 0.6])

        is_double_ticket_available: bool = tickets[m, -1] > 0
        # 10% because 2 tickets out of 24 moves.
        should_use_double_ticket = \
            np.random.choice([True, False], p=[0.1, 0.9]) \
            if is_double_ticket_available \
            else False

        possible_positions = board.get_possible_destinations(positions[m, -1], chosen_transport_type, should_use_double_ticket)
        chosen_position = np.random.choice(possible_positions)

        # Update positions and tickets in the resultschosen_transport_type 
        positions[m+1, -1] = chosen_position
        if chosen_transport_type == TransportType.BLACK or should_use_black_ticket_as_mask:
            tickets[m+1, -2] -= 1
        if should_use_double_ticket:
            tickets[m+1, -1] -= 1

        # 4. Check result
        if positions[m+1, -1] in positions[m+1, :-1]:
            winner = 1 # Detectives won
            break

    return winner, positions, tickets


In [15]:
ITERATIONS = 100
n = 4 # Number of detectives
m = MAXIMUM_MOVES 

result_winner = np.zeros(ITERATIONS, dtype=int) # 1 = detective won; -1 = MrX won.
result_positions = np.zeros((ITERATIONS, m, n+1), dtype=int) # n detectives + Mrx
result_tickets = np.zeros((ITERATIONS, m, 3*n+2), dtype=int) # 3 transport type for n detectives + Black, 2X for Mrx

for i in range(ITERATIONS):
    g_winner, g_positions, g_tickets = generate_single_game(num_detectives=n, num_moves=m)
    result_winner[i] = g_winner
    result_positions[i, :, :] = g_positions
    result_tickets[i, :, :] = g_tickets

In [16]:
result_winner

array([-1, -1, -1,  1,  1,  1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1,  1,
        1,  1, -1, -1, -1, -1,  1, -1, -1, -1,  1,  1, -1, -1, -1, -1, -1,
        1,  1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1,
        1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1,  1, -1,  1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1,  1, -1, -1,
       -1,  1, -1, -1,  1, -1, -1, -1, -1,  1, -1, -1, -1,  1, -1])

In [18]:
result_positions[0]

array([[ 34, 197,  94, 132, 198],
       [ 46, 195,  75, 114, 186],
       [ 47, 197,  94, 126, 198],
       [ 62, 196,  74, 132, 187],
       [ 47, 183,  58, 114, 188],
       [ 34, 196,  44, 132, 159],
       [ 22, 197,  32, 140, 185],
       [ 23, 196,  44, 126, 184],
       [ 22, 197,  58, 114, 185],
       [ 23, 195,  57, 113, 187],
       [ 12, 197,  43, 125, 187],
       [  3,   0,   0,   0, 172],
       [ 12,   0,   0,   0, 171],
       [  0,   0,   0,   0, 170],
       [  0,   0,   0,   0, 185],
       [  0,   0,   0,   0, 140],
       [  0,   0,   0,   0, 154],
       [  0,   0,   0,   0, 153],
       [  0,   0,   0,   0, 154],
       [  0,   0,   0,   0, 139],
       [  0,   0,   0,   0, 130],
       [  0,   0,   0,   0, 131]])

In [19]:
result_tickets[0]

array([[10,  8,  4, 10,  8,  4, 10,  8,  4, 10,  8,  4,  5,  2],
       [10,  7,  4,  9,  8,  4,  9,  8,  4,  9,  8,  4,  4,  2],
       [ 9,  7,  4,  8,  8,  4,  8,  8,  4,  8,  8,  4,  4,  2],
       [ 8,  7,  4,  7,  8,  4,  8,  7,  4,  7,  8,  4,  3,  2],
       [ 7,  7,  4,  6,  8,  4,  8,  6,  4,  6,  8,  4,  3,  2],
       [ 6,  7,  4,  5,  8,  4,  7,  6,  4,  5,  8,  4,  3,  2],
       [ 5,  7,  4,  4,  8,  4,  6,  6,  4,  4,  8,  4,  2,  2],
       [ 4,  7,  4,  3,  8,  4,  5,  6,  4,  3,  8,  4,  1,  2],
       [ 4,  6,  4,  2,  8,  4,  4,  6,  4,  2,  8,  4,  0,  2],
       [ 3,  6,  4,  1,  8,  4,  3,  6,  4,  1,  8,  4,  0,  2],
       [ 2,  6,  4,  0,  8,  4,  2,  6,  4,  0,  8,  4,  0,  1],
       [ 1,  6,  4,  0,  8,  4,  2,  6,  4,  0,  8,  4,  0,  1],
       [ 0,  6,  4,  0,  8,  4,  2,  6,  4,  0,  8,  4,  0,  1],
       [ 0,  6,  4,  0,  8,  4,  2,  6,  4,  0,  8,  4,  0,  1],
       [ 0,  6,  4,  0,  8,  4,  2,  6,  4,  0,  8,  4,  0,  1],
       [ 0,  6,  4,  0,  